# Asistente Para HelpDesk

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer  #Librerias necesarias
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import pickle  #Pickle python es una herramienta para convertir objetos en una secuencia de bytes


def tokenize(sentence): # Declaramos un array tokens 
  tokens=[]  

  for i in range(len(sentence)): 
    tokens.append(sentence[i])  # Separa los parrafos en palabras o 'tokens'
  return tokens




def main():
    Data = pd.read_csv(r"datasetHP.csv", error_bad_lines=False) #Leemos el dataset con pandas
    
    train_text, test_text, train_labels, test_labels = train_test_split(Data["Campo"], #establecemos las variables
                                                                    Data["Etiqueta"],  # de test y train
                                                                    random_state=85)
    
    
    #El tfidVectorizer es un modelo que compara la frecuencia en que se repite una palabra
    Tv = TfidfVectorizer( min_df= 20, max_features=5731, strip_accents='unicode',analyzer='char',ngram_range=(1,18), tokenizer=tokenize)
    train_X = Tv.fit_transform(train_text).toarray() #armamos el array de entrenamiento
    X_test = Tv.transform(test_text).toarray() #array de datos test
  
    
    classNb = MultinomialNB() #inicializamos el modelo predictivo Bayers multinominal
    classNb.fit(train_X, train_labels) #alimentamos el modelo con las variables de entrenaiento
    
    
    predictionNB = classNb.predict(X_test) # predecimos usando las variables de prueba test
 
    
    accuracyNB = accuracy_score(test_labels, predictionNB) # Sacamos el porcentaje de predicción
    print(f"Accuracy NB: {accuracyNB:.4%}") #mostramos en pantalla
    
    filenameTV = 'finalized_TV.sav' 
    pickle.dump(Tv, open(filenameTV, 'wb')) #de este modo guardamos nuestro modelo entrenado en bytes

    filename = 'finalized_ClassNB.sav'
    pickle.dump(classNb, open(filename, 'wb')) #de este modo guardamos nuestro modelo entrenado en bytes

main()
    

Accuracy NB: 97.6971%


In [ ]:
# la variable corpus va a tener todo el texto o cuerpo del ticket

corpus = '''Ubicación : ***** - BXXX01 - *****
Hola buen dia equipo de IT , nos pueden ayudar a eliminar las 
siguientes piezas de las direcciones,
debido a que el operador ingreso las piezas equivocadas.
SKU          cantidad   dirección
MNHW58683     90        Z-0-002-000   
EINL12081      9            
ZAZV12355      4    
                        M-0-005-011
UYEX11111      2         
ttqe99514      55   
Xete00000      11  

Muchas gracias por su ayuda
Saludos
'''    

In [ ]:

#  definimos una funcion que utilizará nuestro modelo entrenado
# para clasificar el texto de la variable corpus
def bot(sentences,classNb,Tv): 

    text1 = sentences 

    examples = [
        text1     #guardamos la palara o token en un array
    ]

    examples_X = Tv.transform(examples) #aplicamos el TfidfVectorizer
    
   
    predict = classNb.predict(examples_X) #ejecutamos el modelo en nuestro token
 
    result = []
    for text, label in zip(examples, predict):
       result ={
           
           label:text  #devolvera un Dir con la etiqueta y el token
                
        }
    return result  


#Declaramos la funcion cleaner, esta nos ayudara a quitar caracteres que no necesitamos
def cleaner(text): 
    #los signos que pueden hacer que nuestro modelo falle
    signal = ['.','#',')','(',',',"'",'"',';','[',']','}','{','/','?','=','+','@','*','+','<','>',':','¡','_']
    #iteramos en el array de signal
    for i in signal:
       t = [char for char in text if char != str(i)] #En resumen el siguiente comprehension list agrega cada 
       text = t                                      #Caracter de la palabra solo si dicho caracter es distinto 
                                                     # a algun signo del array 
    return text

def Display(List): # Funcion que nos despliega la cadena de texto debido a que el cleaner nos devuelve un array
   display = ''  #Inicializamos variable
   for i in range(len(List)): #iteramos en la lista o array
      char = List[i]
      display = display + char #Cadena de caracteres
      
   return display



def mainOut():
    AnswerBool = False
    classNb =  pickle.load(open("finalized_ClassNB.sav", 'rb')) #Con pickle cargamos el obecto
    Tv =  pickle.load(open("finalized_TV.sav", 'rb'))

    while(not AnswerBool): #Bucle mientras que el boolean sea False este se ejecutara continuamente
        try:
             #Desplegamos un pequeño menu, en este caso el bot ejecutaria un POST/GET/PUT en una api y
             #como entrada a esa API vamos a tener los items que va a extraer del texto del ticket
            print('''
                     1 .-  Retirar Error Sistemico - ["SKU","ADDRESS",QUANTITY,"WAREHOUSE"]
                     2 .-  detalles de direcciones - WAREHOUSE,ADDRESS 
                     3 .-  Direcciones Trabadas - ADDRESS,SKU,QUANTITY 
                  ''')

            option = int(input('elige uno:')) #un input que espera del usuario
          
    
            items = {               #Declaramos la variable Dir donde se guardarán nuestro resutados
                     'none':0,       # en este caso NONE serán todos los tokens o palabras que no son utiles
                     'Sku':0,         
                     'Address':0,
                     'Warehouse':0,
                     'Quantity': 0,
                }
             
            for i in corpus.split():  #Empezamos a iterar en la variable corpus

                t = cleaner(i)   #limpiamos nuestra variable de signos indeseados 
                txt = Display(t) #Cleaner nos devuelve un array por lo tanto la funcion display 
                                  #nos devuelve la cadena de texto
                

                items.update(bot(txt.upper(),classNb,Tv)) #ejecutando la funcion bot ingresaremos informacion
                                                          #a la variable Dir
                
                #Luego dependiendo de la opcion que se haya elegido entrará
                #y completará en este caso el input a la API 
                if (option == 1):
                   
                   if items['Address'] != 0 and items['Address'] != '-' and items['Warehouse'] != 0 and items['Quantity'] != 0 and items['Sku'] != 0:
                     #Print luego de validar que todos los campos esten correctos 
                      print('["'+str(items['Sku'])+'","'+str(items['Address'])+'",'+str(items['Quantity'])+',"'+str(items['Warehouse'])+'"],')
                      items['Quantity'] = 0  
                     
                elif (option == 2 ):
                     if items['Address'] != 0 and items['Address'] != '-' and items['Warehouse'] != 0:
                       #Print luego de validar que todos los campos esten correctos   
                       print(str(items['Warehouse'])+','+str(items['Address']))
                       items['Quantity'] = 0 


                elif (option == 3):
                     
                     if items['Address'] != 0 and items['Address'] != '-' and items['Sku'] != 0 and items['Quantity'] != 0:
                         #Print luego de validar que todos los campos esten correctos 
                         print(str(items['Address'])+','+str(items['Sku'])+','+str(items['Quantity'])) 
                         items['Quantity'] = 0

                AnswerBool = True #Sale del ciclo while
                
        except ValueError as e: 
            print(f'Error :{e}' ) #en caso de enviar campos  equivocados


mainOut()


                     1 .-  Retirar Error Sistemico - ["SKU","ADDRESS",QUANTITY,"WAREHOUSE"]
                     2 .-  detalles de direcciones - WAREHOUSE,ADDRESS 
                     3 .-  Direcciones Trabadas - ADDRESS,SKU,QUANTITY 
                  
["MNHW58683","Z-0-002-000",90,"BXXX01"],
["EINL12081","Z-0-002-000",9,"BXXX01"],
["ZAZV12355","Z-0-002-000",4,"BXXX01"],
["UYEX11111","M-0-005-011",2,"BXXX01"],
["TTQE99514","M-0-005-011",55,"BXXX01"],
["XETE00000","M-0-005-011",11,"BXXX01"],


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a83cba6-d8be-468d-939a-6d6572909c44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>